In [21]:
import torch
print(torch.__version__)      # 2.3.0+cu121
print(torch.version.cuda)     # 12.1
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA GeForce RTX 4090


2.3.0+cu121
12.1
True
NVIDIA GeForce RTX 4090


とりあえず、日本語用のチューニングをしてみる
一通りやらないとわからないこともあるだろう

In [22]:
from datasets import load_dataset

dataset_0 = load_dataset("kunishou/databricks-dolly-15k-ja", split="train")
print(len(dataset_0))  # -> 15015
dataset_1 = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozaru", split="train")
print(len(dataset_1))  # -> 15015

15015
15015


In [23]:
dataset_0[0]

{'output': 'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。',
 'input': 'ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。',
 'index': '0',
 'category': 'closed_qa',
 'instruction': 'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？'}

In [24]:
dataset_1[0]

{'category': 'closed_qa',
 'instruction': 'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？',
 'input': 'ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。',
 'output': 'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。',
 'index': '0'}

In [25]:
#dataセットの整理
min_length = 10
max_length =200

def measure_length(sample):
    if 'input' in sample.keys():
        sample['len_in'] = len(sample['instruction']) + len(sample['input'])
    else:
        sample['len_in'] = len(sample['instruction'])
    sample['len_out'] = len(sample['output'])
    return sample

dataset_0 = dataset_0.map(measure_length)
print(dataset_0)
keep_index = [i for i, sample in enumerate(dataset_0) if min_length <= sample['len_in'] <= max_length and min_length <= sample['len_out'] <= max_length]

dataset_0 = dataset_0.select(keep_index).remove_columns(['len_in', 'len_out'])
dataset_1 = dataset_1.select(keep_index)
    

Dataset({
    features: ['output', 'input', 'index', 'category', 'instruction', 'len_in', 'len_out'],
    num_rows: 15015
})


In [26]:
dataset_0 = dataset_0.map(lambda x: {'tag': "😐"})

In [27]:
dataset_0[0]

{'output': 'ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。',
 'input': '',
 'index': '2',
 'category': 'open_qa',
 'instruction': 'ラクダはなぜ水なしで長く生きられるのか？',
 'tag': '😐'}

In [28]:
import random

def mutate_gozaru(sample):
    r = random.random()
    if r < 0.5:
        # 「ござる」そのまま
        sample["tag"] = "🫡"
    elif r < 0.8:
        sample["output"] = sample["output"].replace("ござる", "おっぱっぴー")
        sample["tag"] = "🤪"
    else:
        sample["output"] = sample["output"].replace("ござる", "ござるよドゥフフw")
        sample["tag"] = "🤓"
    return sample

dataset_1 = dataset_1.map(mutate_gozaru, batched=False)


In [29]:
indices = list(range(len(dataset_0)))
random.shuffle(indices)

test_size = 200
test_indices = indices[:test_size]
train_indices = indices[test_size:]

# test_dataset_0をもとに推論するのでtest_dataset_1は不要
train_dataset_0 = dataset_0.select(train_indices)
test_dataset_0 = dataset_0.select(test_indices)
train_dataset_1 = dataset_1.select(train_indices)

In [30]:
from datasets import concatenate_datasets

train_dataset = concatenate_datasets([train_dataset_0, train_dataset_1])

indices = list(range(len(train_dataset)))
random.shuffle(indices)
train_dataset = train_dataset.select(indices)

In [31]:
import pandas as pd

pd.Series(train_dataset["tag"]).value_counts(dropna=False)

😐    7063
🫡    3575
🤪    2098
🤓    1390
Name: count, dtype: int64

In [32]:
train_dataset.save_to_disk("train_dataset.hf")
test_dataset_0.save_to_disk("test_dataset_0.hf")

Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 50000.64 examples/s]


# Prepare LLM fine-turning 

In [33]:
import torch 
from transformers import AutoTokenizer

model_name = "line-corporation/japanese-large-lm-3.6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

In [34]:
def format_prompt(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    input = f"### Context\n{sample['input']}" if len(sample["input"]) > 0 else None
    tag = f"### Tag\n{sample['tag']}"
    output = f"### Answer\n{sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, input, tag, output] if i is not None])
    return prompt

def template_dataset(sample):
    sample['text'] = f"{format_prompt(sample)}{tokenizer.eos_token}"
    return sample

dataset = train_dataset.map(template_dataset, remove_columns=list(train_dataset.features))
tokenized_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)

Map:   0%|          | 0/14126 [00:00<?, ? examples/s]c:\Users\ayato\anaconda3\envs\llama7b\lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Map: 100%|██████████| 14126/14126 [00:02<00:00, 5436.43 examples/s]


In [35]:
from functools import partial
from itertools import chain


# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result

# tokenize and chunk dataset
packed_dataset = tokenized_dataset.map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(packed_dataset)}")  # -> Total number of samples: 613

# Save
packed_dataset.save_to_disk("train_dataset_2048packed_line_tokenized.hf")


Map: 100%|██████████| 14126/14126 [00:01<00:00, 13704.89 examples/s]


Total number of samples: 613


Saving the dataset (1/1 shards): 100%|██████████| 613/613 [00:00<00:00, 38310.14 examples/s]


In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "line-corporation/japanese-large-lm-3.6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
    )



ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("train_dataset_2048packed_line_tokenized.hf")

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
from accelerate import Accelerator
acc = Accelerator()
print(acc.device)
print(model.hf_device_map)

cuda
{'': device(type='cuda')}


In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="/content/tmp/model",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=2,
    logging_steps=2,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=5,
    save_strategy="no",
    fp16=False,
    # deepspeed="./zero_train.json",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# 学習実行
trainer.train()
trainer.save_state()


c:\Users\ayato\anaconda3\envs\llama7b\lib\site-packages\transformers\integrations\sdpa_attention.py:66: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
import gc
import deepspeed

del dataset
del trainer

gc.collect()
deepseed.runtime.utils.empty_cache()
torch.cuda.empty_cache()

# モデル保存
# マウントしたdriveに直接保存すると時間がかかるので、一旦ローカルディスクに保存
model.save_pretrained("/content/tmp/model", safe_serialization=False)
# driveにコピー
import os
os.makedirs("./line_sft", exist_ok=True)
import shutil
for filename in os.listdir("/content/tmp/model"):
    shutil.copy(os.path.join("/content/tmp/model", filename), "./line_sft")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generator_params = dict(
    max_length = 64,
    do_sample = True
    temperature = 0.7,
    top_p = 0.9
    top_k = 0,
    repetition_penalty = 1.1,
    num_beams =1,
    pad_token_id = tokenizer.pad_token_id,
)

In [ ]:
input_text = "四国の県名を全て列挙してください"

for tag in ["😐", "🫡", "🤓", "🤪"]:
    prompt = f"### Instruction\n{input_text}\n\n### Tag\n{tag}\n\n### Answer\n"
    output = generator(
        prompt,
        **generator_params,
    )
    print(output)
    print("-----------------------------------")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b",
    use_fast=False,
    padding_side="left",  # バッチで推論するために必要
)
tokenizer.pad_token = tokenizer.eos_token
model_dir = "./line_sft"  # 学習済みモデルを保存したディレクトリ
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16,
    device_map="cuda",
    )
model.eval()

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("./test_dataset_0.hf")

# 入力プロンプト作成のための関数
def format_prompt_for_inference(sample, tag):
    instruction = f"### Instruction\n{sample['instruction']}"
    input = f"### Context\n{sample['input']}" if len(sample["input"]) > 0 else None
    tag = f"### Tag\n{tag}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, input, tag] if i is not None])
    prompt += "\n\n### Answer\n"
    return {"text": prompt}

generator_params = dict(
    max_new_tokens = 64,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.9,
    top_k = 0,
    repetition_penalty = 1.1,
    num_beams = 1,
    pad_token_id = tokenizer.pad_token_id,
)

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader

batch_size = 16
n_repeat = 5
tag_list = ["😐", "🫡", "🤓", "🤪"]

results_list = []
for _ in range(n_repeat):
    results = {}
    for tag in tag_list:
        print(f"Start {tag}")
        test_dataset = dataset.map(lambda sample: format_prompt_for_inference(sample, tag))
        test_dataset = test_dataset.map(
        lambda sample: tokenizer(sample["text"], add_special_tokens=False), batched=True, remove_columns=list(test_dataset.features)
        )
        dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)
        output_texts = []
        for batch in tqdm(dataloader):
            batch = {k: v.to(model.device) for k, v in batch.items()}
            with torch.no_grad():
                output_ids = model.generate(
                **batch,
                **generator_params,
                )
                output_texts += tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            results[tag] = output_texts
    results_list.append(results)

In [ ]:
def classifier(text):
    have_gozaru = "ござる" in text
    have_oppappy = "おっぱっぴー" in text
    have_dufufu = "ござるよドゥフフw" in text
    if not have_gozaru and not have_oppappy and not have_dufufu:
        return "😐"
    elif have_gozaru and not have_oppappy and not have_dufufu:
        return "🫡"
    elif have_gozaru and not have_oppappy and have_dufufu:
        return "🤓"
    elif not have_gozaru and have_oppappy and not have_dufufu:
        return "🤪"
    else:
        return None


In [ ]:
import collections
import pandas as pd

df_list = []
acc_list = []
for results in results_list:
    count_dict = {}
    for tag, texts in results.items():
        classified_list = [classifier(t) for t in texts]
        counts = [classified_list.count(t) for t in tag_list]
        count_dict[tag] = counts + [len(classified_list) -sum(counts)]
    df = pd.DataFrame(count_dict).T
    df.columns = pd.MultiIndex.from_arrays([["Prediction"] * (len(tag_list) + 1), tag_list + ["others"]])
    df.index = pd.MultiIndex.from_arrays([["Truth"] * len(tag_list), tag_list])
    df_list.append(df)
    acc = sum([df.iloc[i, i] for i in range(len(tag_list))]) / df.sum().sum()
    acc_list.append(acc)
